<h1> Indian Startup Funding </h1>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

In [4]:
ds = pd.read_csv("../input/startup_funding.csv")
ds.head()

In [139]:
ds.shape

<h3> Funding with respect to cities </h3>

In [74]:
city = ds['CityLocation'].value_counts()[:15]
sns.barplot(city.index, city.values, alpha=0.8, color="red")
plt.xticks(rotation='vertical')
plt.xlabel('City location', fontsize=14)
plt.ylabel('Startups funded', fontsize=14)
plt.title("Funding with respect to cities", fontsize=16)
plt.show()

<p>Based on the plot it is safe to assume that bangalore is a major startup hub in India followed by Mumbai and New Delhi<p>
<p> Bangalore has funded more than 600 startups and would be a favorable choice for startups to acquire funding <p>

<h3> Funding with respect to the Industry </h3>

In [73]:
city = ds['IndustryVertical'].value_counts()[:15]
sns.barplot(city.index, city.values, alpha=0.8, color="blue")
plt.xticks(rotation='vertical')
plt.xlabel('Industry', fontsize=14)
plt.ylabel('Startups funded', fontsize=14)
plt.title("Funding with respect to the industry", fontsize=16)
plt.show()

<p> Consumer Internet based startups have acquired the most funding from investors followed by technology ,e-commerce and healthcare<p>
<p> Personally I am amazed by this trend as I have always assumed that there are more startups focussing on Machine learning, Big data analytics, Artificial intelligence and Blockchains<p> 

In [70]:
inv = ds['InvestorsName'].value_counts()
x = inv.values
np.where(x>=10)

<p> Only the first 10 investors are funding more than or equal to 10 startups <p>

<h3> Funding with respect to the Investors </h3>

In [79]:
inv = ds['InvestorsName'].value_counts()[2:10]
sns.barplot(inv.index, inv.values, alpha=0.8, color="green")
plt.xticks(rotation='vertical')
plt.xlabel('Investor', fontsize=14)
plt.ylabel('Startups funded', fontsize=14)
plt.title("Number of startups funded", fontsize=16)
plt.show()

Indian angel network and Ratan Tata are the top investors followed by Kalaari capital

In [124]:
#I had used replace to add the blankspaces in investment type
typ = ds['InvestmentType'].value_counts()
typ

<h3> Funding with respect to the Type of Investment </h3>

In [134]:
sns.barplot(typ.index, typ.values, alpha=0.8, color="black")
plt.xticks(rotation='vertical')
plt.xlabel('Investment type', fontsize=14)
plt.ylabel('Number of startups', fontsize=14)
plt.title("Type of investment", fontsize=16)
plt.show()

For most of the investors **Seed funding** is the favored kind of investment followed by private equity

In [190]:
amt = ds['AmountInUSD']
amt

We use log to make the distribution more analyzable

In [196]:
data = ds["AmountInUSD"].apply(lambda x: float(str(x).replace(",","")))

In [193]:
ds["AmountInUSD"] = ds["AmountInUSD"].apply(lambda x: float(str(x).replace(",","")))
ds['AmountInUSD_log'] = np.log(ds["AmountInUSD"] + 1)
plt.figure(figsize=(8,5))
sns.distplot(ds['AmountInUSD_log'].dropna())
plt.xlabel('log of Amount in USD', fontsize=12)
plt.title("Log Hist of investment in USD", fontsize=16)
plt.show()

In [230]:
ds.head()

<h1> Generate a random startup name based on the given data using LSTM networks <h1> 

In [285]:
data = ds['StartupName'].values
data

In [287]:
data = '\n'.join(data)
data

In [290]:
data = data.replace(" ", "")
data = data.replace("(", "")
data = data.replace(")", "")
data = data.replace("!", "")
data = data.replace(".", "")
data = data.replace("-", "")
data = data.replace("/", "")
data = data.replace("\xa0", "")
data = data.replace("’", "")
data = data.replace("#", "")
data = data.replace("0", "")
data = data.replace("1", "")
data = data.replace("2", "")
data = data.replace("3", "")
data = data.replace("4", "")
data = data.replace("5", "")
data = data.replace("6", "")
data = data.replace("8", "")
data = data.replace("9", "")

In [291]:
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

In [292]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

In [314]:
example = data.split('\n')
example

In [ ]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def initialize_parameters(n_a, n_x, n_y):

    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    
    return a_next, p_t

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    # Initialize x, a and y_hat as empty dictionaries
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    # initialize your loss to 0
    loss = 0
    
    for t in range(len(X)):
        
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # Run one step forward of the RNN
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy term of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

def rnn_backward(X, Y, parameters, cache):
    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    return gradients, a



In [296]:
def clip(gradients, maxValue):

    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in [dWax, dWaa, dWya, db, dby]:
         np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

# GRADED FUNCTION: sample

def sample(parameters, char_to_ix, seed):
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size, 1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a,1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevts entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        # for grading purposes
        np.random.seed(counter+seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):

    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a =  rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]

In [315]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).


    examples = example

    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
                
        # Use the hint above to define one training example (X,Y) (≈ 2 lines)
        index =  j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
                
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result for grading purposed, increment the seed by one. 
      
            print('\n')
        
    return parameters

In [316]:
parameters = model(data, ix_to_char, char_to_ix)

![](http://)<h3> Initially the names make no sense but as as the iteration increases we see some probable company names (Names may be changed as I am training several times)<h3>
<h2> The name IND or INDA comes almost in all iterations indicates that its a common startup name in India, which makes sense <h2>
<ul>
<li>Mabacora </li> <br>
<li>  Votbigo</li><br>
<li>Zystcleri</li><br>
<li>Feagora</li><br>
<li>Inda</li><br>
<li>Voreedom</li><br>
<li>Mevstock and so on....</li>
 
<h2> Thank you



